In [1]:
!pip install fasttext
!pip install fastparquet
!gdown --id '1-2oxSCi9q_iIGHt2JP80dhXL0I4bPoiM'
!gdown --id '1cAthveg1d3MjrKJtMKGzfX3eH8HJ-dQp'
!gdown --id '1-3NwGAk6MW4qDpZnxgIoAzMzFHUazkfN'
!unzip MedNLI_dataset.zip
!pip install stanza
!pip install medialpy
!pip install contractions
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.wv.vectors.npy
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.trainables.syn1neg.npy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246766 sha256=fc0458b508c76f8c9cd386ee949b665c0ad45e69c47d97ca85e9ef40456aba7f
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.5 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  w

In [2]:
# @title imports
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import stanza
import shutil
import numpy as np
from numpy.linalg import norm
import string
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gensim.models import FastText, Word2Vec, KeyedVectors
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import medialpy
from stanza import Pipeline
import contractions

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# @title CODES
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis


class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        for x,y in zip(x_list,y_list):
            #x_tensor = torch.tensor(x,dtype = torch.float32)
            y_tensor = torch.tensor(y,dtype = torch.float32)
            self.samples.append((x[0],x[1],y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


def get_lists(data):
    x_list = []
    y_list = []
    i =0
    for query,answer in zip(data['query'],data['answer']):
        i = i + 1
        if answer == 'entailment':
           y = [1,0,0]
        elif answer == 'neutral':
           y = [0,1,0]
        elif answer == 'contradiction':
           y = [0,0,1]
        else:
           print('should not get here')

        premise,hypothesis = find_pre_and_hyp(query)
        x_list.append((premise,hypothesis))
        y_list.append(y)
    return x_list,y_list


class embedding_layer(nn.Module):
  def __init__(self,bert_model,tokenizer):
    super(embedding_layer, self).__init__()
    self.bert_model = bert_model
    self.tokenizer = tokenizer

  def forward(self, x):
    with torch.no_grad():
         s = tokenizer(x,return_tensors="pt",padding=True).to(device)
         vec = bert_model(**s)['last_hidden_state'].to(device)
    return vec

class PositionalEncoding(nn.Module):
    def __init__(self, seq_len,embedding_size):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.embedding_size = embedding_size
        self.seq_len = seq_len

    def forward(self, x):
        pe = torch.zeros(x.size(0), x.size(1), self.embedding_size).to(device)
        div_term = torch.zeros(x.size(0), 1, self.embedding_size).to(device)
        ks = torch.arange(self.embedding_size).float().to(device)
        values = torch.exp(-torch.log(torch.tensor(1000.0)) * 2 * ks / self.embedding_size).to(device)
        values = values.view(1, 1, -1).to(device)
        div_term = div_term + values
        x = x.reshape([x.shape[0],x.shape[1],1]).to(device)
        pe[:, :, ::2] = torch.sin(x * div_term)[:, :, ::2].to(device)
        pe[:, :, 1::2] = torch.cos(x * div_term)[:, :, 1::2].to(device)
        return self.dropout(pe)

class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_size, heads):
        super().__init__()
        self.embedding_size = embedding_size
        self.heads = heads
        self.head_dim = embedding_size // heads
        assert(self.heads * self.head_dim == self.embedding_size), "Invalid number of heads"
        self.fc_values = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_keys = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_queries = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_out = nn.Linear(heads * self.head_dim, embedding_size).to(device)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)
        values = self.fc_values(values).to(device)
        keys = self.fc_keys(keys).to(device)
        queries = self.fc_queries(queries).to(device)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]).to(device)
        if mask is not None:
            energy = energy.masked_fill(mask.to(device) == 0, float("-1e20")).to(device)
        energy = torch.softmax(energy / (self.embedding_size ** 0.5), dim=3).to(device)
        attention = torch.einsum("nhql,nlhd->nqhd", [energy, values]).to(device)
        attention = attention.reshape(N, query_len, self.heads * self.head_dim).to(device)
        out = self.fc_out(attention)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embedding_size, heads, forward_expansion, p):
        super().__init__()
        self.attention = MultiHeadAttention(embedding_size, heads)
        self.norm1 = nn.LayerNorm(embedding_size)
        self.feed_forward = nn.Sequential(nn.Linear(embedding_size, forward_expansion * embedding_size),
                                          nn.ReLU(),
                                          nn.Linear(forward_expansion * embedding_size, embedding_size))
        self.norm2 = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(p)
    def forward(self, values, keys, queries, mask):
        attention_out = self.attention(values, keys, queries, mask)
        x = self.norm1(attention_out + queries)
        x = self.dropout(x)
        ff_out = self.feed_forward(x)
        out = self.norm2(ff_out + x)
        out = self.dropout(out)
        return out

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_size, num_layers, heads,
                 forward_expansion, max_length, p, device):
        super().__init__()
        self.device = device
        self.word_embedding = embedding_layer(bert_model,tokenizer)
        self.positional_embedding = PositionalEncoding(max_length, embedding_size)
        self.layers = nn.ModuleList([TransformerBlock(embedding_size, heads, forward_expansion, p) for _ in range(num_layers)])
        self.dropout = nn.Dropout(p)

    def forward(self, x, mask):
        mask = None
        pe = self.word_embedding(x)
        N = pe.size(0)
        seq_len = pe.size(1)
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        out = self.dropout((pe + self.positional_embedding(positions)))
        for layer in self.layers:
            out = layer(out, out, out ,mask)
        return out


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, src_pad_idx, embedding_size=768,
                 num_layers=1, forward_expansion=8, heads=8, max_length=100, p=0.1):
        super().__init__()
        self.src_pad_idx = src_pad_idx
        self.encoder = Encoder(src_vocab_size, embedding_size, num_layers, heads,
                               forward_expansion, max_length, p, device)

    def get_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2).to(device)
        return src_mask

    def forward(self, src):
        src_mask = None
        enc_out = self.encoder(src, src_mask).to(device)
        return enc_out


class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.transformer1 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.transformer2 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.fc1 = nn.Linear(768*2, 256).to(device)
        self.fc2 = nn.Linear(256, 3).to(device)

    def forward(self, x_pre,x_hyp):
        out1 = self.transformer1(x_pre)
        out2 = self.transformer2(x_hyp)
        out1 = torch.mean(out1,1)
        out2 = torch.mean(out2,1)
        x = torch.cat((out1,out2),1)
        x = F.relu(self.fc1(self.dropout1(x)))
        x = nn.functional.softmax(self.fc2(self.dropout2(x)),dim=1)
        return x

def get_model_acc(model,data_loader):
    #model.eval()
    #with torch.no_grad():
    predictions = []
    true_labels = []
    for x_pre, x_hyp, y in data_loader:
             outputs = model(x_pre,x_hyp)
             predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
             predictions.extend(predicted_labels)
             l1 = torch.argmax(y, dim=1).cpu().numpy()
             true_labels.extend(l1)
    #model.train()
    return accuracy_score(true_labels, predictions)


def get_inf_from_num(num):
    if num == 0:
       return 'entailment'
    elif num == 1:
       return 'neutral'
    elif num == 2:
       return 'contradiction'

class NNN(nn.Module):
    def __init__(self):
        super(NNN, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.transformer1 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.transformer2 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.fc1 = nn.Linear(768*2, 256).to(device)
        self.fc2 = nn.Linear(256, 2).to(device)

    def forward(self, x_pre,x_hyp):
        out1 = self.transformer1(x_pre)
        out2 = self.transformer2(x_hyp)
        out1 = torch.mean(out1,1)
        out2 = torch.mean(out2,1)
        x = torch.cat((out1,out2),1)
        x = F.relu(self.fc1(self.dropout1(x)))
        x = nn.functional.softmax(self.fc2(self.dropout2(x)),dim=1)
        return x

def get_inference_from_tns(tns):
    infr = np.argmax(tns.cpu().detach().numpy())
    if infr == 0:
       return 'entailment'
    elif infr == 1:
       return 'neutral'
    elif infr == 2:
       return 'contradiction'
    else:
       return 'shouldnt get here'


def get_trustworthy_from_tns(tns):
    infr = np.argmax(tns.cpu().detach().numpy())
    if infr == 0:
       return 'trustworthy'
    elif infr == 1:
       return 'untrustworthy'
    else:
       return 'shouldnt get here'

def get_trust_untrust_dataset(trustmodel,data_loader):
    trustmodel.eval()
    model.eval()
    trust_x_list = []
    untrust_x_list = []
    trust_y_list = []
    untrust_y_list = []
    for x_pre,x_hyp,y in data_loader:
        outputs = model(x_pre,x_hyp)
        outputs_tw = trustmodel(x_pre,x_hyp)
        y = y.to(device)
        for i in range(len(outputs)):
                 p = get_inference_from_tns(outputs[i])
                 t = get_inference_from_tns(y[i])
                 tw = get_trustworthy_from_tns(outputs_tw[i])
                 if tw == 'trustworthy':
                    trust_x_list.append((x_pre[i],x_hyp[i]))
                    trust_y_list.append(y[i].tolist())
                 elif tw == 'untrustworthy':
                    untrust_x_list.append((x_pre[i],x_hyp[i]))
                    untrust_y_list.append(y[i].tolist())
                 else:
                    print('shouldnt get here')
    trust_dataset = CustomDataset(trust_x_list,trust_y_list)
    untrust_dataset = CustomDataset(untrust_x_list,untrust_y_list)
    trust_data_loader = torch.utils.data.DataLoader(trust_dataset, batch_size=32, shuffle=True)
    untrust_data_loader = torch.utils.data.DataLoader(untrust_dataset, batch_size=32, shuffle=True)
    #print('trust num: ' + str(len(trust_y_list)))
    #print('untrust num: ' + str(len(untrust_y_list)))
    #print('trust acc: ' + str(get_model_acc(model,trust_data_loader)))
    #print('untrust acc: ' + str(get_model_acc(model,untrust_data_loader)))
    return trust_dataset,untrust_dataset

In [4]:
test_data = pd.read_parquet('MedNLI_dataset/test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('MedNLI_dataset/train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('MedNLI_dataset/valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
bert_model = AutoModel.from_pretrained("gsarti/biobert-nli").to(device)
src_pad_idx = 0
src_vocab_size = bert_model.config.vocab_size

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [6]:
train_x_list,train_y_list = get_lists(train_data)
test_x_list,test_y_list = get_lists(test_data)
val_x_list,val_y_list = get_lists(valid_data)

In [7]:
train_dataset = CustomDataset(train_x_list,train_y_list)
test_dataset = CustomDataset(test_x_list,test_y_list)
val_dataset = CustomDataset(val_x_list,val_y_list)

In [8]:
bsize = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bsize, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bsize, shuffle=False)

In [9]:
with torch.no_grad():
     model = torch.load('modelw.pth')
     newmodel = torch.load('TrustModelw5.pth')
     print(get_model_acc(model,test_loader))

0.7334739803094233


In [87]:
w2v = Word2Vec.load('w2v_OA_CR_100d.bin')
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| pretrain        | mimic          |
| forward_charlm  | mimic          |
| backward_charlm | mimic          |

INFO:stanza:File exists: /root/stanza_resources/en/tokenize/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/pos/mimic_charlm.pt
INFO:stanza:File exists: /root/stanza_resources/en/lemma/mimic_nocharlm.pt
INFO:stanza:File exists: /root/stanza_resources/en/depparse/mimic_charlm.pt
INFO:stanza:File exists: /root/stanza_resources/en/ner/i2b2.pt
INFO:stanza:File exists: /root/stanza_resources/en/pretrain/mimic.pt
INFO:stanza:File exists: /root/stanza_resources/en/forwar

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | mimic          |
| pos       | mimic_charlm   |
| lemma     | mimic_nocharlm |
| depparse  | mimic_charlm   |
| ner       | i2b2           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [132]:
def extract_svo_rest(sentence):
    doc = nlp(sentence)
    subject = '-'
    verb = '-'
    rest_of_sentence = '-'
    tobe_verbs = ['be','am','is','are','were','was']
    for sent in doc.sentences:
        for word in sent.words:
            if 'subj' in word.deprel:
                subject = word.text
            if word.deprel == 'root':
               if word.upos == 'ADJ':
                  verb = 'is'
               elif word.upos == 'VERB':
                  verb = word.text
        rest_of_sentence = ' '.join([word.text for word in sent.words if word.text != subject and word.text != verb])
    return subject, verb, rest_of_sentence

def get_svos(text):
    sentences = text.split('. ')
    svo_results = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence:
            subject, verb, rest_of_sentence = extract_svo_rest(sentence)
            svo_results.append((sentence, subject, verb, rest_of_sentence))
    return svo_results

In [133]:
def get_cos_similarity(prem,hypt):
    try:
      a = w2v.wv.get_vector(prem)
      b = w2v.wv.get_vector(hypt)
      return (a@b)/(norm(a)*norm(b))
    except:
      return 0

In [134]:
get_cos_similarity('extubated','intubated')

0.860444

In [135]:
def deabbreviation(text):
    try:
       return medialpy.find(text).meaning[0]
    except:
       return text

In [136]:
def preprocess_text(text):
    text = contractions.fix(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    ntokens = []
    for token in tokens:
        ntokens.append(deabbreviation(token).lower())
    stop_words = set(stopwords.words('english'))
    stop_words.remove('no')
    stop_words.remove('not')
    filtered_tokens = [word for word in ntokens if word not in stop_words]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

In [137]:
def check_subject_verb_match(prem_svos,hypt_svos):
    r = False
    for prems, psubjects, pverbs, prests in prem_svos:
        for hypts, hsubjects, hverbs, hrests in hypt_svos:
            if psubjects == hsubjects and pverbs == hverbs:
               r = True
    return r

In [138]:
def check_verb_match(prem_svos,hypt_svos):
    r = False
    for prems, psubjects, pverbs, prests in prem_svos:
        for hypts, hsubjects, hverbs, hrests in hypt_svos:
            if pverbs == hverbs:
               r = True
    return r

In [139]:
def similarity(prem,hypt):
    count = 0
    l = len(word_tokenize(prem)) if len(word_tokenize(prem)) < len(word_tokenize(hypt)) else len(word_tokenize(hypt))
    for hypword in hypt.split():
        for preword in prem.split():
                    if get_cos_similarity(hypword,preword)>=0.8:
                       count +=1
    return count/l

In [162]:
def get_hybrid_answer_sv_match(prem,hypt,dp_answer,sim_score):
    print(sim_score)
    if sim_score > 0.7:
      if (detect_negation(prem) and not detect_negation(hypt)) or (not detect_negation(prem) and detect_negation(hypt)):
          return 'contradiction'
      else:
          return 'entailment'
    else:
      print('default')
      return dp_answer

In [153]:
def is_verb_negated(verb):
    return (get_verb_sentiment(verb)['neg']>=0.9)

In [154]:
def is_s_match_contradiction(a,b,c,d):
    x1 = (not a and b and c and d)
    x2 = (a and not b and c and d)
    x3 = (a and b and not c and d)
    x4 = (a and b and c and not d)
    x5 = (a and not b and not c and not d)
    x6 = (not a and b and not c and not d)
    x7 = (not a and not b and c and not d)
    x8 = (not a and not b and not c and d)
    x9 = ( a and not b and c and not d)
    xx = ( not a and b and not c and d)
    return x1 or x2 or x3 or x4 or x5 or x6 or x7 or x8 or x9 or xx

In [163]:
def get_hybrid_answer_v_match(prem,hypt,dp_answer,sim_score):
    print(sim_score)
    if sim_score > 0.66:
      if (detect_negation(prem) and not detect_negation(hypt)) or (not detect_negation(prem) and detect_negation(hypt)):
          return 'contradiction'
      else:
          return 'entailment'
    else:
      print('default')
      return dp_answer

In [164]:
def get_hybrid_answer_non_match(prem,hypt,prem_verb,hypt_verb,dp_answer,sim_score):
    print(sim_score)
    if sim_score > 0.66:
      a = detect_negation(prem)
      b = detect_negation(hypt)
      c = is_verb_negated(prem_verb)
      d = is_verb_negated(hypt_verb)
      print('prem cx: '+str(a))
      print('hypt cx: '+str(b))
      print('prem verb: '+prem_verb+' '+str(c))
      print('hypt verb: '+hypt_verb+' '+str(d))
      if is_s_match_contradiction(a,b,c,d):
          return 'contradiction'
      else:
          return 'entailment'
    else:
      print('default')
      return dp_answer

In [157]:
def get_reliability_from_num(num):
    if num == 0:
       return 'reliable'
    elif num == 1:
       return 'unreliable'
    else:
       return None

In [158]:
def detect_negation(sentence):
    sentence = contractions.fix(sentence)
    doc = nlp(sentence)
    negations = []
    r = False
    for sentence in doc.sentences:
        for word in sentence.words:
            if (word.text.lower() == "not" or word.text.lower() == "no" or word.text.lower() == "never" or word.text.lower() == "non" or word.text.lower() == "loss" or word.text.lower() == "absence" or word.text.lower() == "low") and word.head > 0:
                head_word = sentence.words[word.head - 1]
                negations.append(head_word.text)
                r = True
    return r

In [159]:
def print_inf(count,wrong_label,corrected_label,uncorrected_label,prem,hypt,hy_answer,dp_answer,ground_truth):
    print('num: '+str(count))
    print('wrong label: '+str(wrong_label))
    print('corrected label: '+str(corrected_label))
    print('uncorrected label: '+str(uncorrected_label))
    print('premise: '+prem)
    print('hypothesis: '+hypt)
    print('hy: '+ hy_answer)
    print('dp: '+ dp_answer)
    print('ground_truth: '+ ground_truth)
    print('=====================================================================================================')

In [160]:
sia = SentimentIntensityAnalyzer()
def get_verb_sentiment(verb):
    return  sia.polarity_scores(verb)

In [172]:
model.eval()
newmodel.eval()
count = 0
wrong_label = 0
corrected_label = 0
uncorrected_label = 0
for x_pre,x_hyp,y in test_loader:
  for i in range(len(x_pre)):
      with torch.no_grad():
          dp_answer = get_inf_from_num(np.argmax(model(x_pre[i],x_hyp[i]).cpu().detach().numpy()))
          ground_truth = get_inf_from_num(np.argmax(y[i].cpu().detach().numpy()))
          dp_trust = get_reliability_from_num(np.argmax(newmodel(x_pre[i],x_hyp[i]).cpu().detach().numpy()))
      if dp_trust == 'unreliable':
          prem = x_pre[i]
          hypt = x_hyp[i]
          prem_svos = get_svos(prem)
          hypt_svos = get_svos(hypt)

          if check_subject_verb_match(prem_svos,hypt_svos):
             sim_score = similarity(preprocess_text(prem),preprocess_text(hypt))
             hy_answer = get_hybrid_answer_sv_match(prem,hypt,dp_answer,sim_score)
             count +=1
             if ground_truth != dp_answer:
                wrong_label += 1
             if ground_truth != dp_answer and hy_answer == ground_truth:
                corrected_label += 1
             if ground_truth == dp_answer and hy_answer != ground_truth:
                uncorrected_label += 1
             print_inf(count,wrong_label,corrected_label,uncorrected_label,prem,hypt,hy_answer,dp_answer,ground_truth)

          elif check_verb_match(prem_svos,hypt_svos):
             sim_score = similarity(preprocess_text(prem),preprocess_text(hypt))
             hy_answer = get_hybrid_answer_v_match(prem,hypt,dp_answer,sim_score)
             count +=1
             if ground_truth != dp_answer:
                wrong_label += 1
             if ground_truth != dp_answer and hy_answer == ground_truth:
                corrected_label += 1
             if ground_truth == dp_answer and hy_answer != ground_truth:
                uncorrected_label += 1
             print_inf(count,wrong_label,corrected_label,uncorrected_label,prem,hypt,hy_answer,dp_answer,ground_truth)
          else:
             sim_score = similarity(preprocess_text(prem),preprocess_text(hypt))
             hy_answer = get_hybrid_answer_non_match(prem,hypt,prem_svos[0][2],hypt_svos[0][2],dp_answer,sim_score)
             count +=1
             if ground_truth != dp_answer:
                wrong_label += 1
             if ground_truth != dp_answer and hy_answer == ground_truth:
                corrected_label += 1
             if ground_truth == dp_answer and hy_answer != ground_truth:
                uncorrected_label += 1
             print_inf(count,wrong_label,corrected_label,uncorrected_label,prem,hypt,hy_answer,dp_answer,ground_truth)

Streaming output truncated to the last 5000 lines.
hypothesis: the patient underwent an EGD
hy: neutral
dp: neutral
ground_truth: entailment
0.0
default
num: 360
wrong label: 128
corrected label: 15
uncorrected label: 4
premise: He had no nausea or vomiting prior the EGD.
hypothesis: the patient has chronic abdominal pain
hy: neutral
dp: neutral
ground_truth: neutral
0.0
default
num: 361
wrong label: 129
corrected label: 15
uncorrected label: 4
premise: Labs notable for WBC 10.5 without bands, Hct 32.2 (prior baseline mid to upper 20s), Cr 0.9, CE neg X 1, and lactate 1.5.
hypothesis: The patient does not have anemia.
hy: contradiction
dp: contradiction
ground_truth: entailment
0.0
default
num: 362
wrong label: 130
corrected label: 15
uncorrected label: 4
premise: Labs notable for WBC 10.5 without bands, Hct 32.2 (prior baseline mid to upper 20s), Cr 0.9, CE neg X 1, and lactate 1.5.
hypothesis: The patient is in renal failure.
hy: entailment
dp: entailment
ground_truth: contradiction
